<a href="https://colab.research.google.com/github/anshupandey/Generative-AI-for-Professionals/blob/main/EY2024/C12_AI_Agents_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding LangChain Agents and Chains

## Chains

Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step. The primary supported way to do this is with LCEL.

## Agents

Agents can be thought of as the chain responsible for deciding what step to take next. This is usually powered by a language model, a prompt, and an output parser.

Different agents have different prompting styles for reasoning, different ways of encoding inputs, and different ways of parsing the output. For a full list of built-in agents see agent types. You can also easily b


## Chains v/s Agents

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.




In [ ]:
# Installing necessary libraries
!pip install langchain-openai langchain langchain-core langchain-community --quiet
!pip install httpx langchain-experimental --quiet
!pip install openai google-search-results tiktoken wikipedia pyowm --quiet

# Chains with LangChain

In [ ]:
api_key = "xxxxxxxxxxxxxxxx"
api_version = "2023-07-01-preview" # "2023-05-15"
azure_endpoint = "https://xxxxxxxxx.openai.azure.com/"
model_name = "gpt-35-turbo"

In [ ]:
import os
os.environ["OPENAI_API_VERSION"] = api_version
os.environ["AZURE_OPENAI_ENDPOINT"] = azure_endpoint
os.environ["AZURE_OPENAI_API_KEY"] = api_key
os.environ["OPENWEATHERMAP_API_KEY"] = "29af1cea50a401d8e624eea4660b3f59"
os.environ["AZURE_OPENAI_API_VERSION"] = api_version

# LangChain AI Agents

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.



## Creating a sample LangChain Agent

# Zero-Shot ReAct Agent

In [ ]:
# Importing necessary modules for the agent
from langchain.agents import load_tools      # Imports the load_tools function for loading various tools in the langchain package
from langchain_openai import AzureChatOpenAI # Imports the ChatOpenAI class to interact with OpenAI's models


In [ ]:
# Initializing a Language Model (LLM) instance
model = AzureChatOpenAI(azure_deployment=model_name,)

In [ ]:
import os
os.environ['TAVILY_API_KEY'] = "tvly-xxxxxxxxxxxxxxxxxxxxxxxxxx"

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)


In [ ]:
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

In [ ]:
model_with_tools = model.bind_tools(tools)

In [ ]:
from langchain_core.messages import HumanMessage


response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

### Implementing Agent with LangChain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)
prompt

In [ ]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "hi!"})

In [ ]:
agent_executor.invoke({"input": "What's the weather in Mumbai?"})

## Implementing a Multitool Agent

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor


prompt = ChatPromptTemplate.from_messages(
    [   ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"), ])

search = TavilySearchResults(max_results=2)
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
# Create Image Gentation model Object

tools = [search, wikipedia]

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "Hi, How are you?"})

In [ ]:
agent_executor.invoke({"input": "What's the current weather in Dubai?"})

In [ ]:
agent_executor.invoke({"input": "what is total area of USA"})

## Thank You